In [2]:
# data에 대한 xpath
#/html/bodt/div/table[1]/tbody/tr[3]/td[1]

import requests
import bs4

page_no = 1
page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}'

source = requests.get(page_url).text
source = bs4.BeautifulSoup(source)
source

dates = source.find_all('td', class_='date')
dates

[<td class="date">2022.12.29</td>,
 <td class="date">2022.12.28</td>,
 <td class="date">2022.12.27</td>,
 <td class="date">2022.12.26</td>,
 <td class="date">2022.12.23</td>,
 <td class="date">2022.12.22</td>]

In [3]:
dates[0].text

'2022.12.29'

In [4]:
# tag를 통해 가져온 다음 텍스트를 가지고옴.
# 데이터는 text 형태로 tag 사이에 있음
# tag 사이에 껴있는 텍스트를 가져올 때 .text를 사용


# 꼭 기억해야할 로직!

date_list = []

for date in dates:
    date_list.append(date.text)

date_list

['2022.12.29',
 '2022.12.28',
 '2022.12.27',
 '2022.12.26',
 '2022.12.23',
 '2022.12.22']

In [ ]:
# tag 규칙이 안 먹힐 때가 있음
# 검사 창에서 보여주는 해당 경로를 우클릭 하면 copy > xpath가 있음

# tag가 어디에 있는지 알려줌.
/html/body/div/table[1]/tbody/tr[3]/td[1]

체결가(종가) 추출 

- 페이지 상의 날짜와 종가정보 전체 추출

In [6]:
prices = source.find_all('td', class_='number_1')
prices
#체결가 뿐만이 아니라 다른것도 number_1로 되어있음
# 그래서 살펴보니 number_1가 4번씩 반복됨.

# 2가지 방법
prices[::4]


[<td class="number_1">291.10</td>,
 <td class="number_1">297.09</td>,
 <td class="number_1">304.22</td>,
 <td class="number_1">302.27</td>,
 <td class="number_1">302.07</td>,
 <td class="number_1">307.48</td>]

In [7]:
price_list =[]
for price in prices[::4]:
    price_list.append(price.text)

price_list

['291.10', '297.09', '304.22', '302.27', '302.07', '307.48']

In [8]:
len(date_list)

6

In [9]:
len(price_list)

6

In [ ]:
# 페이지에서 표의 태그들을 살펴보면 체결가 이후로 전일비/등락률/거래량/거래대금 등도
# 같은 태그를 공유하고 있어, 4개씩 증가하는 것을 알 수 있음

# 이것을 규칙으로 number_1을 추출한 태그는 24개지만, 이 중에서 4배수로 건너뛰면서 추출하면
# 바로 체결가를 가지고 올 수 있음을 확인

In [ ]:
# 맨 뒤까지 뽑아내고 싶음
# 맨 뒤를 클릭하면 맨뒤의 HTML 코드가 나옴

<a href="~~~"
# a href로 시작하는 건 다 링크...라고 생각하면됨 == hyperlink

# 규칙
# 만약 이런 방법으로 맨 뒤까지 소스를 뽑아내고싶다면
# a href= 의 정보가 아닌
# a href= 한 칸 위의 정보까지 뽑아오면 됨

In [12]:
# 해당 tag의 xpath 복사해오기
# (하지만 규칙이 간단해서 xpath를 굳이 사용하지 않아도 됨)

source.find_all('td', class_='pgRR')

[<td class="pgRR">
 <a href="/sise/sise_index_day.naver?code=KPI200&amp;page=701">맨뒤
 				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
 </td>]

In [13]:
# 겉에 씌워진 리스트 제거
source.find_all('td', class_='pgRR')[0]

<td class="pgRR">
<a href="/sise/sise_index_day.naver?code=KPI200&amp;page=701">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>

In [14]:
# <a> </a> 값 뽑아오기
source.find_all('td', class_='pgRR')[0].find_all('a')

[<a href="/sise/sise_index_day.naver?code=KPI200&amp;page=701">맨뒤
 				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>]

In [16]:
# 다시 한 번 리스트 제거
source.find_all('td', class_='pgRR')[0].find_all('a')[0]

<a href="/sise/sise_index_day.naver?code=KPI200&amp;page=701">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>

In [17]:
# herf값 뽑아내
source.find_all('td', class_='pgRR')[0].find_all('a')[0]['href']

'/sise/sise_index_day.naver?code=KPI200&page=701'

In [19]:
last_url = source.find_all('td', class_='pgRR')[0].find_all('a')[0]['href']
last_url

'/sise/sise_index_day.naver?code=KPI200&page=701'

In [21]:
# 마지막 글자를 떼어내기 이해서 규칙을 생성
last_page = int(last_url.split('&page=')[-1])
last_page

# string 값으로 나오므로 int 변환

701

In [22]:
price_list = []
date_list = []

for page_no in range(1, last_page+1):
    page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KOSPI&page={page_no}'

    source = requests.get(page_url).text
    source = bs4.BeautifulSoup(source)

    dates = source.find_all('td', class_='date')

    for date in dates:
        date_list.append(date.text)

    prices = source.find_all('td', class_='number_1')

    for price in prices[::4]:
        price_list.append(price.text)

In [23]:
len(date_list), len(price_list)

(4206, 4206)

In [24]:
6 * 701

4206

In [25]:
import pandas as pd
df = pd.DataFrame({'date': date_list, 'price': price_list})
df

,date,price
0,2022.12.29,"2,236.40"
1,2022.12.28,"2,280.45"
2,2022.12.27,"2,332.79"
3,2022.12.26,"2,317.14"
4,2022.12.23,"2,313.69"
...,...,...
4201,2006.01.02,"1,389.27"
4202,2005.12.29,"1,379.37"
4203,2005.12.28,"1,368.16"
4204,2005.12.27,"1,373.55"


In [26]:
df.isnull().sum()

date     0
price    0
dtype: int64

In [27]:
df.to_excel('kpi200.xlsx', index=False)